In [1]:
# %matplotlib inline

In [2]:
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# !wget https://www.manythings.org/anki/rus-eng.zip
# !unzip rus-eng.zip

In [4]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [5]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Zа-яёъА-ЯЁЪ.!?]+", r" ", s)
    return s

In [25]:
def load_opensubtitles_data(file_path_ru, file_path_en, num_samples=100_000):
    input_texts = []
    with open(file_path_en, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines[:num_samples]:
            en_text = line.strip()
            input_texts.append(normalizeString(en_text))
    target_texts = []
    with open(file_path_ru, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines[:num_samples]:
            ru_text = line.strip()
            target_texts.append(normalizeString(ru_text))
    return input_texts, target_texts

def load_anki_data(file_path, num_samples=100_000):
    input_texts = []
    target_texts = []
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines[:num_samples]:
            en_text, ru_text = line.strip().split('\t')[:-1]
            input_texts.append(normalizeString(en_text))
            target_texts.append(normalizeString(ru_text)) 
    return input_texts, target_texts

In [26]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # lines = open('anki_data/rus.txt', encoding='utf-8').\
    #     read().strip().split('\n')

    # pairs = [[normalizeString(s) for s in l.split('\t')][0:2] for l in lines]
    anki_input_texts, anki_target_texts = load_anki_data('anki_data/rus.txt')
    opensub_input_texts, opensub_target_texts = load_opensubtitles_data('opensubtitles_data/OpenSubtitles.en-ru.ru',
                                                                 'opensubtitles_data/OpenSubtitles.en-ru.en')
    pairs = list(zip(anki_input_texts, anki_target_texts)) + list(zip(opensub_input_texts, opensub_target_texts))
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [27]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH  and \
        p[0].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [28]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'rus', False)
print(random.choice(pairs))

Reading lines...
Read 123924 sentence pairs
Trimmed to 11743 sentence pairs
Counting words...
Counted words:
eng 2663
rus 5399
('i am right for once .', 'на этот раз я прав .')


The Encoder
-----------





In [29]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size,num_layers=1):
        super(EncoderRNN, self).__init__()
        self.num_layers=num_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size,num_layers)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.rnn(output, hidden)
        return output, hidden

    def initHidden(self):
        if str(self.rnn)[:4] == 'LSTM':
            return (torch.zeros(self.num_layers, 1, self.hidden_size, device=device)
                ,torch.zeros(self.num_layers, 1, self.hidden_size, device=device))
        else:
            return torch.zeros(self.num_layers, 1, self.hidden_size, device=device)


The Decoder
-----------




In [30]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size,num_layers=1):
        super(DecoderRNN, self).__init__()
        self.num_layers=num_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size,num_layers)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.rnn(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        if str(self.rnn)[:4] == 'LSTM':
            return (torch.zeros(self.num_layers, 1, self.hidden_size, device=device)
                ,torch.zeros(self.num_layers, 1, self.hidden_size, device=device))
        else:
            return torch.zeros(self.num_layers, 1, self.hidden_size, device=device)

In [31]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [32]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [33]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [34]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, 
               learning_rate=0.01):
    start = time.time()
    print_time = time.time()
    print_iter = 0
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss
        print_iter += 1
    
        if iter % 100 == 0 : 
            if (time.time() - print_time > 30) or iter == n_iters:
                print_time = time.time()
                print_loss_avg = print_loss_total / print_iter
                print_iter = 0
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                             iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [35]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [36]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [37]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [38]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size,1).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words,1).to(device)

trainIters(encoder1, decoder1, 10_000, print_every=1000)

0m 30s (- 2m 28s) (1700 17%) 3.9214
1m 0s (- 2m 3s) (3300 33%) 3.4849
1m 31s (- 1m 31s) (5000 50%) 3.2697
2m 1s (- 1m 2s) (6600 66%) 3.1544
2m 32s (- 0m 35s) (8100 81%) 3.0345
3m 3s (- 0m 5s) (9700 97%) 2.9514
3m 9s (- 0m 0s) (10000 100%) 2.9736


In [19]:
evaluateRandomly(encoder1, decoder1)

> you re so dirty .
= вы такие грязные !
< ты все . . <EOS>

> i m going to need some more money .
= мне понадобится еще немного денег .
< я все с в . . . <EOS>

> i m satisfied with my salary .
= я довольна своеи зарплатои .
< я все с с томом . <EOS>

> i m very busy these days .
= я очень занят в эти дни !
< я не очень . . . <EOS>

> you re bugging me .
= ты меня раздражаешь .
< ты меня . . <EOS>

> we re against that .
= мы против этого .
< мы все . . <EOS>

> they are exhausted .
= они обессилены .
< они все . . <EOS>

> i m sure tom is bored .
= я уверен что тому скучно .
< я уверен что том том . . <EOS>

> we are teachers .
= мы учителя .
< мы все . <EOS>

> i m not married anymore .
= я больше не женат .
< я не не . . . <EOS>



Я попробовал разные RNN с разным количеством слоев
Однослойная GRU быстрее обучается.
Увеличение количесва слоев, в данном случае, снижает скорость обучения
и не дает заметных плюсов.
Качество перевода немного увеличивается по мере обучения, 
но нормального перевода данная модель не сделает, сколько не учи.
Не ясно, что с переобучением
Не представляю, как програмно оценить адекватность перевода